In [1]:
!pip install tetgen
!pip install panel
!apt-get install -qq xvfb
!pip install pyvista panel -q
!pip install -q piglet pyvirtualdisplay
!pip install pyacvd
# !pip install ipyvtklink
# !pip install itkwidgets

     |████████████████████████████████| 1.9 MB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 5.2 MB/s eta 0:00:01     |███████████████████             | 798 kB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 5.9 MB/s eta 0:00:01     |███████▊                        | 798 kB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 158 kB 6.3 MB/s eta 0:00:01
     |████████████████████████████████| 59.5 MB 23 kB/s  eta 0:00:011    |███████▌                        | 13.8 MB 8.8 MB/s eta 0:00:06     |█████████▉                      | 18.2 MB 9.6 MB/s eta 0:00:05     |██████████▊                     | 20.0 MB 9.6 MB/s eta 0:00:05     |█████████████████████▉          | 40.5 MB 9.0 MB/s eta 0:00:03     |███████████████████████         | 42.9 MB 9.0 MB/s eta 0:00:02     |█████████████████████████▌      | 47.5 MB 10.3 MB/s eta 0:00:02
     |████████████████████████████████| 3.1 MB 7.7 MB/s eta 0:00:01
     |████████████████████████████████| 495 kB 1

In [2]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(600, 400))
display.start()

In [3]:
import meshio
import os
import tetgen
import pyvista
import numpy as np
import pyacvd

In [4]:
def refine_surface_mesh(mesh, subdiv=6, num_cluster=2000):
    # First, refine surface mesh:
    clus = pyacvd.Clustering(mesh)
    clus.subdivide(subdiv)
    clus.cluster(num_cluster)
    remesh = clus.create_mesh()
    return remesh

In [5]:
def create_volume_mesh(mesh, mindihedral=20):
    # Next, convert surface mesh to volume mesh:
    tet = tetgen.TetGen(mesh)
    tet.tetrahedralize(order=1, mindihedral=mindihedral) 
    return tet.grid

In [6]:
def get_submesh(mesh, axis, cutoff):
    cell_center = mesh.cell_centers().points
    mask = cell_center[:, axis] < cutoff
    cell_ind = mask.nonzero()[0]
    submesh = mesh.extract_cells(cell_ind)
    return submesh

In [75]:
def visualise_mesh(grid, subgrid=None, title=None):
    pyvista.start_xvfb(wait=0.05)
    p = pyvista.Plotter(notebook=True, window_size=[960,480]) #
    title = "Breast Mesh" if title is None else title
    p.add_text(title, name="title", position="upper_edge")
    if subgrid is not None:
        p.add_mesh(grid, style="wireframe", color="k") 
        p.add_mesh(subgrid, lighting=True, show_edges=True) 
    else: 
        p.add_mesh(grid, show_edges=True, edge_color='k', lighting=False)
    
    p.show_axes()
    # p.show_bounds()
    p.show_grid()
    viewer = p.show(jupyter_backend='panel', return_viewer=True)
    return viewer

In [81]:
def transform_mesh(mesh):
    mesh.rotate_x(90)
    mesh.rotate_z(90)
    mesh.scale(1000)
    min_pt = np.min(mesh.points, axis=0)
    mesh.translate(-1*min_pt)
    return mesh

In [77]:
mesh = pyvista.read('breast_closed.obj')
visualise_mesh(mesh, title='Original Surface Mesh')

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

In [7]:
remesh = refine_surface_mesh(mesh, subdiv=6, num_cluster=500)

/usr/local/lib/python3.9/dist-packages/pyvista/core/dataset.py:1192: PyvistaDeprecationWarning: Use of `point_arrays` is deprecated. Use `point_data` instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pyvista/core/dataset.py:1332: PyvistaDeprecationWarning: Use of `cell_arrays` is deprecated. Use `cell_data` instead.
  warnings.warn(


In [83]:
remesh = pyvista.read('breast_remeshed.obj')
visualise_mesh(remesh, title='Refined Surface Mesh')

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

In [82]:
transformed = transform_mesh(remesh)
visualise_mesh(transformed, title='Transformed Surface Mesh')

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

In [87]:
# Write surface meshes to new .obj files:
pyvista.save_meshio('breast.obj', transformed)

In [86]:
vol = create_volume_mesh(transformed, mindihedral=23)
subvol = get_submesh(vol, axis=1, cutoff=80)
visualise_mesh(vol, subvol, title='Volume Mesh with Cross-Section')

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)